# Parallel Request Demo for Hugging Face API (via Docker + FastAPI)
Created on 2025-06-01

This notebook sends multiple **parallel requests** to the locally running Docker container serving a Hugging Face sentiment analysis model using **FastAPI**. 

In [1]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

API_URL = "http://localhost:8000/predict"


In [2]:
texts = [
    "Today has been a lovely day!",
    "This is the worst experience ever.",
    "Not bad, but could be better.",
    "Absolutely fantastic!",
    "Terrible service and rude behavior.",
    "Everything was perfect.",
    "I'm not sure how I feel about this.",
    "Awful. Just awful.",
    "It was okay, nothing special.",
    "Best decision I made!"
]


In [3]:
def send_request(text):
    try:
        response = requests.post(API_URL, json={"text": text})
        return {
            "text": text,
            "status_code": response.status_code,
            "response": response.json()
        }
    except Exception as e:
        return {
            "text": text,
            "status_code": "Error",
            "response": str(e)
        }


In [4]:
start_time = time.time()
results = []

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(send_request, text) for text in texts]
    for future in as_completed(futures):
        results.append(future.result())

end_time = time.time()
duration = end_time - start_time


In [5]:
df = pd.DataFrame(results)
df["duration_secs"] = round(duration, 2)
df

,text,status_code,response,duration_secs
0,I love this app!,200,"{'label': 'POSITIVE', 'score': 0.9999}",0.26
1,Absolutely fantastic!,200,"{'label': 'POSITIVE', 'score': 0.9999}",0.26
2,Terrible service and rude staff.,200,"{'label': 'NEGATIVE', 'score': 0.9994}",0.26
3,This is the worst experience ever.,200,"{'label': 'NEGATIVE', 'score': 0.9998}",0.26
4,"Not bad, but could be better.",200,"{'label': 'NEGATIVE', 'score': 0.8016}",0.26
5,"It was okay, nothing special.",200,"{'label': 'NEGATIVE', 'score': 0.9821}",0.26
6,Awful. Just awful.,200,"{'label': 'NEGATIVE', 'score': 0.9998}",0.26
7,Everything was perfect.,200,"{'label': 'POSITIVE', 'score': 0.9999}",0.26
8,I'm not sure how I feel about this.,200,"{'label': 'NEGATIVE', 'score': 0.9992}",0.26
9,Best decision I made!,200,"{'label': 'POSITIVE', 'score': 0.9996}",0.26
